In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../../modules')

In [2]:
import numpy as np
import factors

# Probabilities
Probability theory is the way to represent how much we believe in certain models of the world, grounded in a set of sensible axioms. We take a group of models, e.g the arbitary set $A$, and write out how much we believe in each model, $A_i$. The probability of a model in a set of models $A$ is a number $P(A_i)$, often shortened to $P(A)$ which then refers to a function of each element in $A$. We can write joint probabilities $P(A,B)$ which describes the probability of one model and another, as well as conditional probabilities $P(A|B)$ which denotes the probability of one model given another (in the above case $A$ given $B$). We can write the joint equivalently as $P(A \cap B)$, as well as the probability that either $A$, $B$ or both occur as $P(A\cup B)$. There are three axioms of probability:

### The Kolmogorov Axioms
**1:** Probabilities are always greater than 0. <br>
**2:** The sum of probabilities of all models in the set is 1.<br>
**3:** The union of probabilities of disjoint sets add. This is the same as saying if event $A$ and event $B$ cannot both occur (mutually exclusive) then the probability of $A$ or $B$ is the probability of $A$ plus the probability of $B$. Think a dice with numbers 1 to 6. The probability of 1 or 2 is the probability of 1 plus the probability of 2.<br>
From the above axioms the main laws of probability can be derived:

### Laws of probability:
**1:** The sum rule: $P(A\cup B) = P(A) + P(B) - P(A\cap B)$ <br>
**2:** The product rule: $P(A\cap B) = P(A|B)P(B)$ <br>
**3:** The law of toal probability: $\sum_B P(A\cap B)=P(A)$ <br>
With these laws and the axioms it is easy to see how probabilities can be expanded and contracted into different formulas.

### Examples:
Say we have two events, $R$ the probability it will rain today (which has two possible options, "yes" and "no") and $C$ the probability I will wear a raincoat (also "yes", "no"). <br>
Say it rains every fourth day. <br>
Also say when it rains I wear a raincoat $90\%$ of the time and when it doesn't I wear it $20\%$ of the time. <br>
We then know: <br> <br>
$P(R=yes)=0.25$ <br>
$P(C=yes|R=yes)=0.9$ <br>
$P(C=yes|R=no)=0.2$ <br> <br>
By knowing these probabilitites I can also get the opposite using the fact that probabilitites must sum to 1: <br> <br>
$P(R=no)=0.75$ <br>
$P(C=no|R=yes)=0.1$ <br>
$P(C=no|R=yes)=0.8$ <br> <br>
Thus we know $P(R)$ and $P(C|R)$, therefore as $P(R,C)=P(C|R)P(R)$: <br> <br>
$P(C=yes,R=yes)=0.25\times0.9=0.225$ <br>
$P(C=yes,R=no)=0.75\times0.2=0.15$ <br>
$P(C=no,R=yes)=0.25\times0.1=0.025$ <br>
$P(C=no,R=no)=0.75\times0.8=0.6$ <br> <br>
As expected, the sum of the joint probabilitites is 1: <br> $0.225+0.15+0.025+0.6=1$ <br> <br>
In general it is very useful to be able to break down a probability by expanding it with the product rule.

# Factors
One way of representing a joint probability is with a factor. A factor is essentially a table which maps every possible event/model to a number. For instance, the above joint distribution can be written:

In [3]:
names = ["I'm wearing a raincoat","It's raining"]
pos_values = [2,2]
rain_factor = factors.Factor(names,pos_values)
rain_factor.set([0,0],0.225)
rain_factor.set([0,1],0.15)
rain_factor.set([1,0],0.025)
rain_factor.set([1,1],0.6)
print(rain_factor)

I'm wearing a raincoat  It's raining  Values (10 dp)
0                       0             0.225
0                       1             0.15
1                       0             0.025
1                       1             0.6



To view this factor code I wrote, look in the modules page.

Interestingly a factor doesn't have to be normalized so it sums to 1 to represent a distribution, that can come later. For instance the below alternative gives the same relative probabilities:

In [4]:
rain_factor_unnorm = factors.Factor(names,pos_values)
rain_factor_unnorm.set([0,0],22.5)
rain_factor_unnorm.set([0,1],15)
rain_factor_unnorm.set([1,0],2.5)
rain_factor_unnorm.set([1,1],60)
print(rain_factor_unnorm)

I'm wearing a raincoat  It's raining  Values (10 dp)
0                       0             22.5
0                       1             15.0
1                       0             2.5
1                       1             60.0



And can be later normalized to give the original values:

In [5]:
# conditioning with no argument normalizes
print(factors.condition(rain_factor_unnorm))

I'm wearing a raincoat  It's raining  Values (10 dp)
0                       0             0.225
0                       1             0.15
1                       0             0.025
1                       1             0.6



You can condition a factor, which means to renormalize the probabilitites with respect to one variable:

In [6]:
prob_C_given_R = factors.condition(rain_factor,"It's raining")
print(prob_C_given_R)

I'm wearing a raincoat  It's raining  Values (10 dp)
0                       0             0.9
0                       1             0.2
1                       0             0.1
1                       1             0.8



And you can marginalize, which means to apply the total probability law, $\sum_B P(A,B)=P(A)$

In [7]:
prob_R = factors.marginalize(rain_factor,"I'm wearing a raincoat")
print(prob_R)

It's raining  Values (10 dp)
0             0.25
1             0.75



You can also multiply factors together; if $\phi_1(A,B)$ is the factor of sets $A$ and $B$, and $\phi_2(B,C)$ is the factor for sets $B$ and $C$, then the factor product is just $\phi_1(A,B)\phi_2(B,C)$. In certain cases, this is just the product rule. If $\phi_1=P(A|B)$ and $\phi_2=P(B)$ then the factor product is just $P(A|B)P(B)=P(A,B)$, as for the two factors above this gives the joint probability:

In [8]:
joint_R_C = factors.product(prob_C_given_R,prob_R)
print(joint_R_C)

I'm wearing a raincoat  It's raining  Values (10 dp)
0                       0             0.225
0                       1             0.15
1                       0             0.025
1                       1             0.6



However! It is important to note that the factor product isn't just the product rule. It can be much more general than that. At the end of the day factors are just functions we make up to represent correlations and independences between models:

# Graphs, Conditional Independence
It is often true but not always that the probability of one event and another are independent, or independent given another event. Here independence just means $P(A|B)=P(A)$, whereas conditional independence means $P(A|B,C)=P(A|C)$, if $A$ is independent from $B$ given $C$. A directed graph is used to represent these ideas: <br>

Variables are considered as nodes in a graph. If a variable is dependent on another it is given a directed arrow from the variable it depends on to itself. If a variable is independent of all other variables it has no incoming links. <br>
#### Examples:

<img src="Images/basic_directed_pgms.jpg" width="400" align="left">

The above are the simple pgms for 3 variables. They can be derived from the product rule for probabilitites: <br>
**E.g:** <br>
$\begin{aligned}
P(A,B,C)  &= P(C|A,B)P(A,B) \\
          &= P(C|A,B)P(B|A)P(A) \\
\end{aligned}$ <br>
That describes exactly the first pgm above. $A$ and $B$ determine $C$, and $A$ determines $B$. So graph *#1* is another way of constructing the joint distribution. <br> <br>
If we make assumptions about our variables, then some elements in the above can be changed.
For instance, if we say $P(C|A,B)=P(C|B)$ (aka that $C$ is independent of $A$ given $B$), then the joint can be written: <br>
$P(A,B,C)  = P(C|B)P(B|A)P(A)$ <br>
Which is what the *#2* graph shows. <br> <br>
Alternatively if we say $P(C|A,B)=P(C|A)$ (aka that $C$ is independent of $B$ given $A$), then the joint can be written: <br>
$P(A,B,C)  = P(C|A)P(B|A)P(A)$ <br>
This correspondes to the *#3* graph. <br> <br>
Finally, if we say $P(B|A)=P(B)$ then we get the formula: <br>
$P(A,B,C)  = P(C|A,B)P(B)P(A)$ <br>
Which correspondes to graph *#4*

As note before, we can represent joint and conditional distributions with factors, so we can also describe graphs with factors as well. A factor is drawn as a square. In these cases each factor just represents a conditional probability table.

<img src="Images/basic_factor_graph_pgms.jpg" width="400" align="left">

### Markov Blanket
The markov blanket refers to the minimum number of variables you need to know in order to make one variable completely independent of all others. This is just the number of variables which are 1 step away from the selected variable in the factor graph. For instance, in graph *#3* above the variable $C$ only has 1 variable, $A$ which is 1 step away. So knowing $A$ means the probability of $C$ is completely independent of the probabilitites of all other variables (just $B$ in this case). In other words knowing the parents of a node, the children, and the other parents of all children is enough. 

### An extra note on factors
Factors have the power to describe much more than conditional distributions. If all we believe about the world is that $A$ is independent of $C$ given $B$, then we cannot be sure whether $A$ and $C$ depend on $B$ or whether $C$ depends on $B$ which depends on $A$. Luckily, factor multiplication describes exactly the family of possible (unnormalized) functions which could give rise to the independence property. If $\phi_1(A,B)$ and $\phi_2(B,C)$ are the two factors then the joint is defined $\phi_3(A,B,C)=\phi_1(A,B)\phi_2(B,C)$, which must have the property that $A$ and $C$ and independent given $B$, as only when variables are independent can their probabilities be simply multiplied.

### The importance of model assumptions
When we say that a variable is independent or conditionally independent of another we reduce the number of variables in our model. This is very important, as with a large number of variables comes very large joint factors. For instance, say you want to drop off something at a friends house. There are three variables you care about, whether they are home, wether it is raining, and whether you have enough petrol in your car. You want to decide whether to try go to your friends house, which involves knowing the probabilitites of the above. In total there are 8 different possibilities. That means learning/storing 7 numbers (as the very last one can be infered). However, if the probabilitity you are out of petrol is independent of whether your friend is home and it it/isn't raining, then the joint with 8 variables can be replaced with the joint distribution for whether your friend is home and it is raining, along with the probability you have enough petrol. <br>
**E.g**

In [9]:
home_raining_factor = factors.Factor(["home","raining"],[2,2])
home_raining_factor.set([0,0],0.35)
home_raining_factor.set([0,1],0.15)
home_raining_factor.set([1,0],0.2)
home_raining_factor.set([1,1],0.3)
print(home_raining_factor)

petrol_factor = factors.Factor(["petrol"],[2])
petrol_factor.set([0],0.8)
petrol_factor.set([1],0.2)
print(petrol_factor)

joint_factor = factors.product(home_raining_factor,petrol_factor)
print(joint_factor)

home  raining  Values (10 dp)
0     0        0.35
0     1        0.15
1     0        0.2
1     1        0.3

petrol  Values (10 dp)
0       0.8
1       0.2

home  raining  petrol  Values (10 dp)
0     0        0       0.28
0     0        1       0.07
0     1        0       0.12
0     1        1       0.03
1     0        0       0.16
1     0        1       0.04
1     1        0       0.24
1     1        1       0.06



This requires 3 variables in the first factor to be stored, and only 1 for the petrol factor. So in total 4 variables, rather than 7.